In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip -q install transformers

In [ ]:
import os
os.chdir('/content/drive/My Drive/1011: Term Project/Collab Notebooks/George')

Native DialoGPT - before finetuning

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")



## Model configuration

In [ ]:
#https://huggingface.co/transformers/v2.0.0/examples.html#language-model-fine-tuning
#https://colab.research.google.com/drive/15wa925dj7jvdvrz8_z3vU7btqAFQLVlG#scrollTo=CjZaN5ilgd-z

"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = '/content/drive/MyDrive/1011: Term Project/Collab Notebooks/DialoGPT_personas/DialoGPT-medium/'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-medium'
        self.config_name = 'microsoft/DialoGPT-medium'
        self.tokenizer_name = 'microsoft/DialoGPT-medium'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = False
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 1
        self.per_gpu_eval_batch_size = 1
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.seed = 42
        self.local_rank = -1

args = Args()

# Load and preprocess data

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/1011: Term Project/Collab Notebooks/DialoGPT_personas/train.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/1011: Term Project/Collab Notebooks/DialoGPT_personas/test_contexted.csv')

train_data = train_data.drop(columns=['Unnamed: 0'])
val_data = valid_data.drop(columns=['Unnamed: 0'])

# Empty strings can't be processed 
train_data = train_data.dropna()
val_data = val_data.dropna()

## Helper functions

In [ ]:
# Convert data to feed to model
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class SeinfeldDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.max_len - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Caching and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return SeinfeldDataset(tokenizer, args, df_val if evaluate else df_trn)

def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        shutil.rmtree(checkpoint)

## Training loop

In [ ]:
def collate(examples: List[torch.Tensor]):
    if tokenizer._pad_token is None:
        return pad_sequence(examples, batch_first=True)
    return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=1, collate_fn=collate, drop_last = True)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay,},
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

    # Check if saved optimizer or scheduler states exist
    if (os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))):
      
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
        model = torch.nn.DataParallel(model)

    # Train
    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        except ValueError:
            logger.info(" Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch")

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            
            #KW: TO AVOID RUNNING OUT OF CUDA MEMORY
            inputs.detach()
            labels.detach()
            # ---

            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))
    print(perplexity)

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        for key in sorted(result.keys()):
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

## Main function

In [ ]:
# Main function

def main(df_trn, df_val):
    args = Args()

    # Setup CUDA
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=True,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    # # Training
    # train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
    # global_step, tr_loss = train(args, train_dataset, model, tokenizer)

    # # Create output directory if needed
    # os.makedirs(args.output_dir, exist_ok=True)

    # # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # # They can then be reloaded using `from_pretrained()`
    # model.save_pretrained(args.output_dir)
    # tokenizer.save_pretrained(args.output_dir)
    # torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

    # # Load a trained model and vocabulary that you have fine-tuned
    # model = AutoModelWithLMHead.from_pretrained(args.output_dir)
    # tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
    # model.to(args.device)

    # Validation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(train_data, val_data)

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the TF 2.0 model and are newly initialized: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer


tensor(186.5527)


{'perplexity_': tensor(186.5527)}

# Dialogue

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/1011: Term Project/Collab Notebooks/data/test.csv')

In [ ]:
# import random 
# import pandas as pd
# import re

# from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForCausalLM
# import torch

def talking_gpt2_personas(test_dict, model, name1, name2, num_lines):
  """
  This function takes are arguments the below inputs and processes a dialogue
  between two characters from Sienfield of a choosing.

  Inputs:
    (DataFrame) test_dict: lines from Sienfield that the gpt2s were not trained on 
    (gpt2)      model: gpt2 model with personas
    (str)       name1: name of Character A (capitalize for distinction from model)
    (str)       name2: name of Character B (capitalize for distinction from model)
    (int)       num_lines: number of lines of dialouge to generate
    (tokenizer) t
  Outputs: 
    Dialogue iteraction between Character A and Character B using the 2 gpt2 
    models.
    (list)      references: lines that actually follow the initial input
    (list)      candidates: lines that are predicted to follow the initial input
  """

  test_personas = {"JERRY": ["your persona: i make a living telling jokes. \nyour persona: i like to watch sports. \nyour persona: i am a fan of cartoons. \nyour persona: women find me quirky and charming."],
                  "GEORGE": ["your persona: i am vulnerable and slightly neurotic. \nyour persona: i am short, stocky, and slow-witted. \nyour persona: i am dishonest. \nyour persona: i have eccentric behavior."],
                  "ELAINE": ["your persona: i am intelligent. \nyour persona: i am funny. \nyour persona: i am assertive and confident. \nyour persona: i am edgy and superficial."],
                  "KRAMER": ["your persona: i have eccentric behavior. \nyour persona: i am unemployed. \nyour persona: i always tell the truth. \nyour persona: i have interesting ideas."]
                  }

  #tokenizer = t

  # Initalize lists to keep track of references and candidates
  references = []
  candidates = []
  
  # Select only lines from Character A from the test DataFrame
  charA_df = test_dict[test_dict.Character == name1]

  # Randomly select an input statement for the dialogue from Character A's lines
  # This is supplemented with Character B's persona 
  rand_idx = random.choice(list(charA_df.index))
  input = test_personas[name2][0] + '\n ' + charA_df['Dialogue'][rand_idx]
  #print(input)
  print(f"{name1}: {charA_df['Dialogue'][rand_idx]}")

  # Append to references list the next 'step' number of lines 
  i = rand_idx
  for step in range(num_lines):
    i += 1
    references.append(test_dict['Dialogue'][i].split(' '))

  for step in range(num_lines):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    #new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # generate a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000,pad_token_id=tokenizer.eos_token_id,  
      no_repeat_ngram_size=3, do_sample=True, top_k=100, top_p=0.7, temperature = 0.8)
    
    # At first step & even steps, use the input from the test dictionary to generate a new line
    # for Character B
    if step % 2 == 0:
      print("{}: {}".format(name2, tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))) 
      candidates.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split(' '))
      input = test_personas[name1][0] + '\n ' + tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    else:
      print("{}: {}".format(name1, tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))) 
      candidates.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True).split(' '))
      input = test_personas[name2][0] + '\n ' + tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
  
  return references, candidates

In [ ]:
references, candidates = talking_gpt2_personas(test, model, "GEORGE", "JERRY", 5)


GEORGE: i ' m writing .
JERRY: That's just you being a self righteous narcissist.
GEORGE: That's a good summary of the whole thing.
JERRY: That's what I thought at first. But then I realized it's just a play on words.
GEORGE: Oh yeah. I'm pretty sure it's a play off of that.
JERRY: I'd say I'm a fan, but I'm more of a fan than I am a person.


In [ ]:
references, candidates = talking_gpt2_personas(test, model, "GEORGE", "JERRY", 5)

GEORGE: it ' s about the show . it ; no , it was ;
JERRY: No one can take that as a compliment.
GEORGE: I'm pretty sure they're all just joking around
JERRY: you can tell the difference between joking around and serious about something
GEORGE: I see you've read the dictionary definition of cynicism.
JERRY: I've read your dictionary definition.


In [ ]:
references, candidates = talking_gpt2_personas(test, model, "ELAINE", "GEORGE", 5)

ELAINE: right , so , i called my friend , you know - the one who set us up - i found out , he ' s a bad - breaker - upper .
GEORGE: I like that one, that's how I'll keep it from being used as a punchline.
ELAINE: I'm a bit of a narcissist, so that works, right?
GEORGE: I have a very similar one. It's similar to yours, but not the same as yours.
ELAINE: I've got the same one
GEORGE: That's what I was looking for! Thank you so much!


In [ ]:
references, candidates = talking_gpt2_personas(test, model, "ELAINE", "GEORGE", 5)

ELAINE: they have no idea when they ' re going back to florida ?
GEORGE: I can't be bothered.
ELAINE: That's a great summary of my whole persona.
GEORGE: I'm just starting out, but I'm thinking I should really start writing.
ELAINE: I agree with the first part of your persona.
GEORGE: I feel like I'm not the only one who noticed this...


In [ ]:
references, candidates = talking_gpt2_personas(test, model, "JERRY", "ELAINE", 5)

JERRY: i can ' t believe your a friend of mine .
ELAINE: That's a little too much
JERRY: I love to watch cartoons!
ELAINE: I'm pretty sure that's how most of my friendships are described.
JERRY: What about you?
ELAINE: I'm a little bit awkward


In [ ]:
references, candidates = talking_gpt2_personas(test, model, "JERRY", "KRAMER", 5)

JERRY: spring . of course .
KRAMER: I can't stop laughing at this.
JERRY: And now I have to go back and watch the video again because I can see you're laughing now.
KRAMER: I thought I was on r theredpill for a second
JERRY: Haha, yeah I thought that was a pretty accurate summary of the two.
KRAMER: I'm a big fan of this, I wish I had more time to work on it, but I am terrible at it.
